In [1]:
### Assigned by ###
# Gal Israeli #####
# Yulia Kuderko ###
# Ram Michaeli ####
###################

import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, normalize
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import random
import math
from sklearn.metrics import r2_score
import numpy as np
import sklearn.metrics as metrics


%matplotlib inline

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("Volcano_after_cleaning.csv")
df.head()

,volcanoName,eruptionCertainty,startDate,maxVEI,evidenceMethod,latitude,longitude,volcanoHeight,country,primaryVolcanoType,population5km,population10km,population30km,population100km
0,Ambae,1,2021 Dec 5,2,Historical Observations,-15.389,167.835,1496,Vanuatu,Shield,73,1279,4326,73308
1,Pinatubo,1,2021 Nov 30,4,Historical Observations,15.130,120.350,1486,Philippines,Stratovolcano,725,3808,1148684,21875048
2,Iliwerung,1,2021 Nov 28,1,Historical Observations,-8.532,123.573,583,Indonesia,Complex,968,4318,79569,665081
3,Taal,1,2021 Nov 15,2,Historical Observations,14.002,120.993,311,Philippines,Caldera,717090,717090,2380326,24814047
4,Turrialba,1,2021 Nov 3,2,Historical Observations,10.025,-83.767,3340,Costa Rica,Stratovolcano,1891,6867,672428,3390544


## Predict Primary Volcano Type Based On Volcano Height, Latitude, Longitude, Max VEI

In [3]:
X = df[["volcanoHeight","latitude","longitude","maxVEI"]]
y = df[["primaryVolcanoType"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)
scaler = MinMaxScaler(feature_range=(-1,1))
X_train_scaled = scaler.fit_transform(X_train.values)
print(f"Test size is: {0.3}")
print("Initial amount of samples: #{}".format(X.shape[0]))
print("Number of training samples: #{}".format(X_train.shape[0]))
print("Number of test samples: #{}".format(X_test.shape[0]))

Test size is: 0.3
Initial amount of samples: #10636
Number of training samples: #7445
Number of test samples: #3191


In [4]:
print("Classifier = 'KNN':")
parameters = {'n_neighbors':range(3,103,2) }
knn = KNeighborsClassifier()
clf = GridSearchCV(knn, parameters,scoring=make_scorer(metrics.accuracy_score, greater_is_better=True))
clf.fit(X_train_scaled, y_train.values)
params_KNN = list(clf.best_params_.values())[0]
print("best parameter set is:",clf.best_params_," and its score was",clf.best_score_)
print("--------------------------------------------")

print("Classifier = 'Decision Tree':")
parameters = {'max_depth':range(1,20),'min_samples_split':range(10,50,5) }
decisionTree = tree.DecisionTreeClassifier()
clf = GridSearchCV(decisionTree, parameters,scoring=make_scorer(metrics.accuracy_score, greater_is_better=True))
clf.fit(X_train_scaled, y_train.values)
params_DT_max_depth = list(clf.best_params_.values())[0]
params_DT_min_samples_split = list(clf.best_params_.values())[1]
print("best parameter set is:",clf.best_params_," and its score was",clf.best_score_)
print("--------------------------------------------")

print("Classifier = 'Random Forest':")
parameters = {'n_estimators':range(50,551,50) }
rf = RandomForestClassifier()
clf = GridSearchCV(rf, parameters,scoring=make_scorer(metrics.accuracy_score, greater_is_better=True))
clf.fit(X_train_scaled, y_train.values)
params_RF = list(clf.best_params_.values())[0]
print("best parameter set is:",clf.best_params_," and its score was",clf.best_score_)
print("--------------------------------------------")

Classifier = 'KNN':
best parameter set is: {'n_neighbors': 3}  and its score was 0.9305574210879787
--------------------------------------------
Classifier = 'Decision Tree':
best parameter set is: {'max_depth': 19, 'min_samples_split': 10}  and its score was 0.9701813297515111
--------------------------------------------
Classifier = 'Random Forest':
best parameter set is: {'n_estimators': 250}  and its score was 0.9875083948959033
--------------------------------------------


In [5]:
clf1 = KNeighborsClassifier(n_neighbors=params_KNN)
clf2 = tree.DecisionTreeClassifier(max_depth= params_DT_max_depth, min_samples_split= params_DT_min_samples_split)
clf3 = RandomForestClassifier(n_estimators=params_RF,bootstrap=True)
clf4 = GaussianNB()
algNames=["KNN", "Decision Tree", "Random Forest", "Naive Bayes"]
bestCLF = None
bestACC = 0
X_test_scaled = scaler.fit_transform(X_test.values)
for i,clf in enumerate([clf1,clf2,clf3,clf4]):
    clf.fit(X_train_scaled, y_train.values)
    y_pred = clf.predict(X_test_scaled)
    y_pred_train = clf.predict(X_train_scaled)
    print(algNames[i])
    currentACC = metrics.accuracy_score(y_true = y_train, y_pred = y_pred_train)
    print('Accuracy on training data = ', metrics.accuracy_score(y_true = y_train, y_pred = y_pred_train))
    print('Accuracy on test data = ', metrics.accuracy_score(y_true = y_test, y_pred = y_pred))
    print("--------------------------------------------")
    if currentACC > bestACC:
        bestACC = currentACC
        bestCLF=clf
print("The best CLF is: "+str(bestCLF))

KNN
Accuracy on training data =  0.9717931497649429
Accuracy on test data =  0.5803823252898778
--------------------------------------------
Decision Tree
Accuracy on training data =  0.9869711215580926
Accuracy on test data =  0.4713256032591664
--------------------------------------------
Random Forest
Accuracy on training data =  1.0
Accuracy on test data =  0.6176747101222188
--------------------------------------------
Naive Bayes
Accuracy on training data =  0.6703828072531901
Accuracy on test data =  0.6270761516765904
--------------------------------------------
The best CLF is: RandomForestClassifier(n_estimators=250)


In [6]:
for i in range(0,4):
    lat = (random.uniform(-90, 90))
    long = (random.uniform(-180, 180))
    vHeight = int(random.uniform(-4200, 6879))
    maxVEI = int(random.uniform(0, 7))
    
    print("[Lat , Long , Height , MaxVEI] \n"+str(round(lat,2))+" , "+str(round(long,2))+" , "+str(vHeight)+" , "+str(maxVEI))

    scaledList = scaler.transform([[vHeight,lat,long,maxVEI]])
    print("The predicted primary volcano type is: "+str(bestCLF.predict(scaledList))+"\n")

[Lat , Long , Height , MaxVEI] 
-25.59 , 144.48 , 760 , 1
The predicted primary volcano type is: ['Caldera']

[Lat , Long , Height , MaxVEI] 
48.42 , 131.15 , -3620 , 3
The predicted primary volcano type is: ['Submarine']

[Lat , Long , Height , MaxVEI] 
-68.81 , -55.93 , -1178 , 5
The predicted primary volcano type is: ['Submarine']

[Lat , Long , Height , MaxVEI] 
-81.76 , -179.16 , 6381 , 6
The predicted primary volcano type is: ['Stratovolcano']



## Predict SO2_Altitude_AVG

In [7]:
df_emission = pd.read_csv("Volcano_Emission.csv")

df_emission.sort_values("SO2_Altitude")

names = list(df_emission.volcanoName.value_counts().keys())

df_emission[df_emission['volcanoName']=='Etna'].SO2_Altitude.mean()

combined = pd.DataFrame()
SO2_Mass_kt_AVG = list()
SO2_Altitude_AVG = list()
for name in names:
    SO2_Mass_kt_AVG.append(df_emission[df_emission['volcanoName']==name].SO2_Mass_kt.mean())
    SO2_Altitude_AVG.append(df_emission[df_emission['volcanoName']==name].SO2_Altitude.mean())
data= \
    {
        'names':names,
        'SO2_Mass_kt_AVG':SO2_Mass_kt_AVG,
        "SO2_Altitude_AVG":SO2_Altitude_AVG
    }
combined = pd.DataFrame.from_dict(data)


for i,j in df.iterrows():
    if j['volcanoName'] not in names:
        df.drop(i,inplace=True)
        
df['SO2_Mass_kt_AVG'] = None
df['SO2_Altitude_AVG'] = None

for i in range(0,df.shape[0]):
    df['SO2_Altitude_AVG'].iloc[i] = float(combined.loc[combined['names']==df['volcanoName'].iloc[i]]['SO2_Altitude_AVG'])
    df['SO2_Mass_kt_AVG'].iloc[i] = float(combined.loc[combined['names']==df['volcanoName'].iloc[i]]['SO2_Mass_kt_AVG'])

In [8]:
cols = ['SO2_Mass_kt_AVG','SO2_Altitude_AVG']
df[cols] = df[cols].apply(pd.to_numeric)
numericDF=df.select_dtypes('number')
numericDF.SO2_Altitude_AVG=numericDF.SO2_Altitude_AVG.astype(int)

In [9]:
numericDF.head()

,eruptionCertainty,maxVEI,latitude,longitude,volcanoHeight,population5km,population10km,population30km,population100km,SO2_Mass_kt_AVG,SO2_Altitude_AVG
1,1,4,15.130,120.350,1486,725,3808,1148684,21875048,18194.000000,19
4,1,2,10.025,-83.767,3340,1891,6867,672428,3390544,3.000000,5
7,1,0,19.421,-155.287,1222,3122,3122,8495,169550,32.774286,5
11,1,2,55.417,-161.894,2493,0,0,0,3002,33.500000,11
12,1,1,18.130,145.800,570,0,0,0,0,320.000000,20


In [10]:
X = numericDF.drop('SO2_Altitude_AVG', axis =1)
y = numericDF[['SO2_Altitude_AVG']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)
scaler = MinMaxScaler(feature_range=(-1,1))
X_train_scaled = scaler.fit_transform(X_train.values)
print(f"Test size is: {0.3}")
print("Initial amount of samples: #{}".format(X.shape[0]))
print("Number of training samples: #{}".format(X_train.shape[0]))
print("Number of test samples: #{}".format(X_test.shape[0]))

Test size is: 0.3
Initial amount of samples: #4481
Number of training samples: #3136
Number of test samples: #1345


In [11]:
print("Classifier = 'KNN':")
parameters = {'n_neighbors':range(3,103,2) }
knn = KNeighborsClassifier()
clf = GridSearchCV(knn, parameters,scoring=make_scorer(metrics.accuracy_score, greater_is_better=True))
clf.fit(X_train_scaled, y_train.values)
params_KNN = list(clf.best_params_.values())[0]
print("best parameter set is:",clf.best_params_," and its score was",clf.best_score_)
print("--------------------------------------------")

print("Classifier = 'Decision Tree':")
parameters = {'max_depth':range(1,20),'min_samples_split':range(10,50,5) }
decisionTree = tree.DecisionTreeClassifier()
clf = GridSearchCV(decisionTree, parameters,scoring=make_scorer(metrics.accuracy_score, greater_is_better=True))
clf.fit(X_train_scaled, y_train.values)
params_DT_max_depth = list(clf.best_params_.values())[0]
params_DT_min_samples_split = list(clf.best_params_.values())[1]
print("best parameter set is:",clf.best_params_," and its score was",clf.best_score_)
print("--------------------------------------------")

print("Classifier = 'Random Forest':")
parameters = {'n_estimators':range(50,551,50) }
rf = RandomForestClassifier()
clf = GridSearchCV(rf, parameters,scoring=make_scorer(metrics.accuracy_score, greater_is_better=True))
clf.fit(X_train_scaled, y_train.values)
params_RF = list(clf.best_params_.values())[0]
print("best parameter set is:",clf.best_params_," and its score was",clf.best_score_)
print("--------------------------------------------")

Classifier = 'KNN':
best parameter set is: {'n_neighbors': 3}  and its score was 0.9359034528997654
--------------------------------------------
Classifier = 'Decision Tree':
best parameter set is: {'max_depth': 17, 'min_samples_split': 10}  and its score was 0.9948983634535093
--------------------------------------------
Classifier = 'Random Forest':
best parameter set is: {'n_estimators': 50}  and its score was 0.9984051036682615
--------------------------------------------


In [12]:
clf1 = KNeighborsClassifier(n_neighbors=params_KNN)
clf2 = tree.DecisionTreeClassifier(max_depth= params_DT_max_depth, min_samples_split= params_DT_min_samples_split)
clf3 = RandomForestClassifier(n_estimators=params_RF,bootstrap=True)
clf4 = GaussianNB()
algNames=["KNN", "Decision Tree", "Random Forest", "Naive Bayes"]
bestCLF = None
bestACC = 0
X_test_scaled = scaler.fit_transform(X_test.values)
for i,clf in enumerate([clf1,clf2,clf3,clf4]):
    clf.fit(X_train_scaled, y_train.values)
    y_pred = clf.predict(X_test_scaled)
    y_pred_train = clf.predict(X_train_scaled)
    print(algNames[i])
    currentACC = metrics.accuracy_score(y_true = y_train, y_pred = y_pred_train)
    print('Accuracy on training data = ', metrics.accuracy_score(y_true = y_train, y_pred = y_pred_train))
    print('Accuracy on test data = ', metrics.accuracy_score(y_true = y_test, y_pred = y_pred))
    print("--------------------------------------------")
    if currentACC > bestACC:
        bestACC = currentACC
        bestCLF=clf
print("The best CLF is: "+str(bestCLF))

KNN
Accuracy on training data =  0.9719387755102041
Accuracy on test data =  0.9360594795539033
--------------------------------------------
Decision Tree
Accuracy on training data =  0.9974489795918368
Accuracy on test data =  0.9970260223048327
--------------------------------------------
Random Forest
Accuracy on training data =  1.0
Accuracy on test data =  0.9985130111524163
--------------------------------------------
Naive Bayes
Accuracy on training data =  0.4882015306122449
Accuracy on test data =  0.5003717472118959
--------------------------------------------
The best CLF is: RandomForestClassifier(n_estimators=50)
